In [442]:
# SET UP AND IMPORTS 
import pandas as pd 

df = pd.read_excel('dataset/processed/psl.xlsx')
start_place = 'PSL'
#start_legs contain the legs with departure place as start_place
start_legs = df[df['Departure Place'] == start_place].sort_values('Departure Time', ascending=True)
#main_legs contains all other legs 
main_legs = df[df['Departure Place'] != start_place]
display(start_legs.head())
display(main_legs.head())

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time
0,1,13200,PSL,NH,KLKV,13500,3.5,300
14,15,15600,PSL,NH,KLKV,15900,3.5,300
3,4,16200,PSL,NH,KLKV,16800,3.5,600
643,645,18000,PSL,KRKM-MJ,TVM,23700,39.0,5700
691,693,18000,PSL,MKD-KLD-PLKDA-NTA-TVM-MC-SKRM-CHPY,PCD,27900,66.5,9900


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time
65,66,41700,KLKV,NH,PSL,42300,3.5,600
66,67,43500,KLKV,NH,PSL,44100,3.5,600
67,68,45900,KLKV,NH,PSL,46500,3.5,600
68,69,47100,KLKV,NH,PSL,47700,3.5,600
69,70,46800,KLKV,NH,PSL,47400,3.5,600


In [443]:
#SUPPORTING FUNCTIONS 
MIN_DUTY = 25200 #setting the minimum duty as 7:00 hrs 
# The ideal duty is 7:30 hrs with a 15 min break at the start and end . Then total 8:00 
MAX_DUTY = 30600
min_terminal_gap = 300
max_terminal_gap = 900
steering_duty = 0
 # setting the maximum duty as 8:30 hrs
def generateTrip(start_leg):
    stack = [] # contain the legs in the trip 


In [444]:
'''trip = [{"current_leg": start_legs.iloc[0].reset_index(drop=True),
                "temp_set": start_legs.iloc[1].reset_index(drop=True).drop(index=0).reset_index(drop=True),
                "duty":  start_legs.iloc[0]['Running Time'],
                }]
display(trip)'''

'trip = [{"current_leg": start_legs.iloc[0].reset_index(drop=True),\n                "temp_set": start_legs.iloc[1].reset_index(drop=True).drop(index=0).reset_index(drop=True),\n                "duty":  start_legs.iloc[0][\'Running Time\'],\n                }]\ndisplay(trip)'

In [445]:
'''def Steering_Duty(leg):
    global trip 
    duty = trip[-1]['duty'] + leg['Running Time'][0]
    last_arrival = trip[-1]['current_leg']['Arrival Time act'][0] 
    new_departure = leg['Departure Time'][0] 
    terminal_gap = new_departure - last_arrival
    return (terminal_gap, duty)'''


"def Steering_Duty(leg):\n    global trip \n    duty = trip[-1]['duty'] + leg['Running Time'][0]\n    last_arrival = trip[-1]['current_leg']['Arrival Time act'][0] \n    new_departure = leg['Departure Time'][0] \n    terminal_gap = new_departure - last_arrival\n    return (terminal_gap, duty)"

In [446]:
leg = start_legs.iloc[0]
def generate_temp_set(leg):
    # Creating a copy of the main legs
    temp_set = df[(df['Departure Place'] == leg['Arrival Place']) & (df['Departure Time'] > leg['Arrival Time act'])]
    # Sorting and resetting index
    temp_set = temp_set.sort_values('Departure Time', ascending=True).reset_index(drop=True)
    temp_set['Terminal Gap'] = temp_set['Departure Time'] - leg['Arrival Time act']
    temp_set['Duty'] = leg['Running Time'] + temp_set['Running Time'].cumsum()
    #display(temp_set)
    temp_set = temp_set[
        (temp_set['Terminal Gap'] >= min_terminal_gap) & (temp_set['Terminal Gap'] < max_terminal_gap) #|
        #(temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)  
    ]
    return temp_set
result_temp_set = generate_temp_set(leg)
display(result_temp_set)

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap,Duty
0,124,13800,KLKV,NH,TVM,17700,33.7,3900,300,4200


In [447]:
# Sample DataFrames
data1 = {
    'Departure Time': ['03:40:00'],
    'Departure Place': ['PSL'],
    'Arrival Place': ['KLKV'],
    'Running Time': ['00:05:00'],
}

data2 = {
    'Departure Time': ['03:50:00'],
    'Departure Place': ['KLKV'],
    'Arrival Place': ['TVM'],
    'Running Time': ['01:05:00'],
}

data3 = {
    'Departure Time': ['05:05:00'],
    'Departure Place': ['TVM'],
    'Arrival Place': ['KNVLA'],
    'Running Time': ['01:00:00'],
}

data4 = {
    'Departure Time': ['06:15:00'],
    'Departure Place': ['KNVLA'],
    'Arrival Place': ['UDA'],
    'Running Time': ['01:45:00'],
}
 
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)
df4 = pd.DataFrame(data4)
 
stack = [
    {"current_leg": df1, "temp_set": []},
    {"current_leg": df2, "temp_set": []},
    {"current_leg": df3, "temp_set": []},
    {"current_leg": df4, "temp_set": []},
]

In [448]:
# Function to display and convert current_legs to a DataFrame
def getTrip():
    current_legs = []
    route = [x['current_leg'] for x in stack ]
    result_df = display(pd.concat(route).reset_index(drop=True))
    return result_df

result_dataframe = getTrip()
display(result_dataframe)

,Departure Time,Departure Place,Arrival Place,Running Time
0,03:40:00,PSL,KLKV,00:05:00
1,03:50:00,KLKV,TVM,01:05:00
2,05:05:00,TVM,KNVLA,01:00:00
3,06:15:00,KNVLA,UDA,01:45:00


None